# Pre-processing Arizona Allocation data for WaDEQA upload.
Date Updated: 04/06/2020

Purpose:  To pre-process the Arizona data into one master file for simple DataFrame creation and extraction

Notes:
- We are going to use all AZ groundwater for now.  They don't tradtionaly track priority date, but we want to incldue the volume and site info.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

## Groundwater

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData/Groundwater"
os.chdir(workingDir)

WellRegistry_Input = "WELLS_wellRegistry_input.csv"
GWSI_Input = "GWSI_SITES_input.csv"

In [3]:
#Groundwater Dataframes creation
df_WR = pd.read_csv(WellRegistry_Input)
df_WR['REG_ID'] = df_WR['REGISTRY_I'].astype(str)
df_GWSI = pd.read_csv(GWSI_Input)

# Merge together into one
df_WR_GWSI = pd.merge(df_WR, df_GWSI, left_on='REG_ID', right_on='REG_ID', how='left')
df_WR_GWSI.head(3)

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,31,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,OID__x,PROGRAM,REGISTRY_I,OWNER_NAME,RGR_PUMP_D,WELLTYPE,WELL_TYPE_,DLIC_NUM,APPROVED,INSTALLED,WELL_DEPTH_x,WATER_LEVE,CASING_DEP,CASING_DIA,CASING_TYP,PUMP_TYPE,PUMP_POWER,PUMPRATE,TESTEDRATE,DRAW_DOWN,COMPLETION,DRILL_LOG,WELL_CANCE,CADASTRAL,COUNTY,WATERSHED,BASIN_NAME,SUBBASIN_N,AMA,QUAD_CODE,WHOLE_TOWN,HALF_TOWNS,NORTHSOUTH,WHOLE_RANG,HALF_RANGE,EASTWEST,SECTION,QUARTER_16,QACRE160DI,QUARTER_40,QACRE40DIR,QUARTER_10,QACRE10DIR,UTM_X_METE,UTM_Y_METE,APPLICATIO,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,ZIP4,REG_ID,OID__y,OBJECTID,SITE_ID,LOCAL_ID,WELL_TYPE,DD_LAT,DD_LONG,WELL_ALT,WATER_USE,WELL_DEPTH_y,CASE_DIAM,LASTWLDATE,WL_DTW,WL_ELEV
0,0,55,60000,AR SACATON LLC,NO,NON-EXEMPT,NON-EXEMPT,0,NaN,NaN,1790,257.0,60,20.0,,,NO POWER CODE LISTED,0,0,0,,,Y,D05005035ABA,PINAL,SANTA CRUZ RIVER,PINAL AMA,ELOY,PINAL,D,5,0,S,5,0,E,35,A,NE,B,NW,A,NE,423912.1,3646244.0,1/1/1900 0:00:00,1150 N 7TH AVE,PO BOX 5747,TUCSON,AZ,,,60000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,1/24/2003 0:00:00,7/2/1984 0:00:00,1000,520.0,1000,12.0,,,ELECTRIC MOTOR 16 - 100 HP,200,400,75,,,,B05002008CCB,MARICOPA,AGUA FRIA RIVER,PHOENIX AMA,WEST SALT RIVER VALLEY,PHOENIX,B,5,0,N,2,0,W,8,C,SW,C,SW,B,NW,361297.1,3739323.0,2/25/1985 0:00:00,"1114 AVENUE OF THE AMERICAS, 39TH FLOOR",,NEW YORK,NY,,,60001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,55,82721,"KEYES,G",NO,NON-EXEMPT,NON-EXEMPT,63,NaN,NaN,0,0.0,0,0.0,,,NO POWER CODE LISTED,0,0,0,,,Y,B02009022000,MARICOPA,LOWER GILA RIVER,HARQUAHALA INA,HARQUAHALA,HARQUAHALA INA,B,2,0,N,9,0,W,22,0,,0,,0,,298366.4,3708927.0,10/15/1979 0:00:00,7765 E FOOTHILL DR S,,PARADISE VALLEY,AZ,,,82721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Creating long and lat values from data.  
# Need to convert from UTM to WGS 84.
# I believe AZ is consiered WGS 84 / UTM zone 12N - EPSG:32612.

from pyproj import Proj
myProj = Proj(proj='utm',zone=12, ellps='WGS84', preserve_units=False)
long, lat = myProj(df_WR_GWSI['UTM_X_METE'].values, df_WR_GWSI['UTM_Y_METE'].values, inverse=True)
df_WR_GWSI['in_Latitude'] = lat
df_WR_GWSI['in_Longitude'] = long
df_WR_GWSI = df_WR_GWSI.replace(np.nan, '')  # Replaces NaN values with blank.
df_WR_GWSI.head(3)

,OID__x,PROGRAM,REGISTRY_I,OWNER_NAME,RGR_PUMP_D,WELLTYPE,WELL_TYPE_,DLIC_NUM,APPROVED,INSTALLED,WELL_DEPTH_x,WATER_LEVE,CASING_DEP,CASING_DIA,CASING_TYP,PUMP_TYPE,PUMP_POWER,PUMPRATE,TESTEDRATE,DRAW_DOWN,COMPLETION,DRILL_LOG,WELL_CANCE,CADASTRAL,COUNTY,WATERSHED,BASIN_NAME,SUBBASIN_N,AMA,QUAD_CODE,WHOLE_TOWN,HALF_TOWNS,NORTHSOUTH,WHOLE_RANG,HALF_RANGE,EASTWEST,SECTION,QUARTER_16,QACRE160DI,QUARTER_40,QACRE40DIR,QUARTER_10,QACRE10DIR,UTM_X_METE,UTM_Y_METE,APPLICATIO,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,ZIP4,REG_ID,OID__y,OBJECTID,SITE_ID,LOCAL_ID,WELL_TYPE,DD_LAT,DD_LONG,WELL_ALT,WATER_USE,WELL_DEPTH_y,CASE_DIAM,LASTWLDATE,WL_DTW,WL_ELEV,in_Latitude,in_Longitude
0,0,55,60000,AR SACATON LLC,NO,NON-EXEMPT,NON-EXEMPT,0,,,1790,257.0,60,20.0,,,NO POWER CODE LISTED,0,0,0,,,Y,D05005035ABA,PINAL,SANTA CRUZ RIVER,PINAL AMA,ELOY,PINAL,D,5,0,S,5,0,E,35,A,NE,B,NW,A,NE,423912.1,3646244.0,1/1/1900 0:00:00,1150 N 7TH AVE,PO BOX 5747,TUCSON,AZ,,,60000,,,,,,,,,,,,,,,32.951858,-111.814054
1,1,55,60001,"SFI GRAND VISTA, LLC",YES,NON-EXEMPT - NON-SERVICE,NON-EXEMPT,611,1/24/2003 0:00:00,7/2/1984 0:00:00,1000,520.0,1000,12.0,,,ELECTRIC MOTOR 16 - 100 HP,200,400,75,,,,B05002008CCB,MARICOPA,AGUA FRIA RIVER,PHOENIX AMA,WEST SALT RIVER VALLEY,PHOENIX,B,5,0,N,2,0,W,8,C,SW,C,SW,B,NW,361297.1,3739323.0,2/25/1985 0:00:00,"1114 AVENUE OF THE AMERICAS, 39TH FLOOR",,NEW YORK,NY,,,60001,,,,,,,,,,,,,,,33.784972,-112.498131
2,2,55,82721,"KEYES,G",NO,NON-EXEMPT,NON-EXEMPT,63,,,0,0.0,0,0.0,,,NO POWER CODE LISTED,0,0,0,,,Y,B02009022000,MARICOPA,LOWER GILA RIVER,HARQUAHALA INA,HARQUAHALA,HARQUAHALA INA,B,2,0,N,9,0,W,22,0,,0,,0,,298366.4,3708927.0,10/15/1979 0:00:00,7765 E FOOTHILL DR S,,PARADISE VALLEY,AZ,,,82721,,,,,,,,,,,,,,,33.500894,-113.170597


In [5]:
# Create output dataframe for ground water
columnslist = [
    ### Method Info ###
    "in_ApplicableResourceTypeCV",
    
    ### Water Source Info ###
    "in_WaterSourceUUID",
    "in_WSGeometry",
    "in_GNISFeatureNameCV",
    "in_WaterQualityIndicatorCV",
    "in_WaterSourceName",
    "in_WaterSourceNativeID",
    "in_WaterSourceTypeCV",
    
    ### Site Info ###
    "in_SiteUUID",
    "in_CoordinateAccuracy",
    "in_CoordinateMethodCV",
    "in_County",
    "in_EPSGCodeCV",
    "in_Geometry",
    "in_GNISCodeCV",
    "in_HUC12",
    "in_HUC8",
    "in_Latitude",
    "in_Longitude",
    "in_NHDNetworkStatusCV",
    "in_NHDProductCV",
    "in_PODorPOUSite",
    "in_SiteName",
    "in_SiteNativeID",
    "in_SitePoint",
    "in_SiteTypeCV",
    "in_StateCV",
    "in_USGSSiteID",
    
    ### AllocationAmount_fact Info ###
    "in_AllocationApplicationDate",
    "in_AllocationAssociatedConsumptiveUseSiteIDs",
    "in_AllocationAssociatedWithdrawalSiteIDs",
    "in_AllocationBasisCV",
    "in_AllocationChangeApplicationIndicator",
    "in_AllocationCommunityWaterSupplySystem",
    "in_AllocationCropDutyAmount",
    "in_AllocationExpirationDate",
    "in_AllocationFlow_CFS",
    "in_AllocationLegalStatusCV",
    "in_AllocationNativeID",
    "in_AllocationOwner",
    "in_AllocationPriorityDate",
    "in_AllocationSDWISIdentifierCV",
    "in_AllocationTimeframeEnd",
    "in_AllocationTimeframeStart",
    "in_AllocationTypeCV",
    "in_AllocationVolume_AF",
    "in_BeneficialUseCategory",
    "in_CommunityWaterSupplySystem",
    "in_CropTypeCV",
    "in_CustomerTypeCV",
    "in_DataPublicationDate",
    "in_DataPublicationDOI",
    "in_ExemptOfVolumeFlowPriority",
    "in_GeneratedPowerCapacityMW",
    "in_IrrigatedAcreage",
    "in_IrrigationMethodCV",
    "in_LegacyAllocationIDs",
    "in_MethodUUID",
    "in_OrganizationUUID",
    "in_PopulationServed",
    "in_PowerType",
    "in_PrimaryUseCategory",
    "in_SiteUUID",
    "in_VariableSpecificUUID",
    "in_WaterAllocationNativeURL",
    "in_WaterSourceUUID"
]

dfground = pd.DataFrame(columns=columnslist, index=df_WR_GWSI.index)

In [6]:
#############################################################################################
#Method
dfground['in_ApplicableResourceTypeCV'] = 'Groundwater'

#WaterSource
dfground['in_WaterSourceTypeCV'] = 'groundwater/well'
dfground['in_WaterSourceName'] = "Unspecified"
                                    
#Site
dfground['in_County'] = df_WR_GWSI['COUNTY']
dfground['in_Latitude'] = df_WR_GWSI['in_Latitude']
dfground['in_Longitude'] = df_WR_GWSI['in_Longitude']
dfground['in_PODorPOUSite'] = 'POD'
dfground['in_SiteName'] = "Unspecified"
dfground['in_SiteNativeID'] = df_WR_GWSI['REGISTRY_I']
dfground['in_SiteTypeCV'] = df_WR_GWSI['WELL_TYPE_']

#AllocationAmount_fact
dfground['in_AllocationFlow_CFS'] = df_WR_GWSI['PUMPRATE']
dfground['in_AllocationNativeID'] = df_WR_GWSI['REGISTRY_I']
dfground['in_AllocationOwner'] = df_WR_GWSI['OWNER_NAME']
dfground['in_AllocationPriorityDate'] = ''
dfground['in_AllocationTimeframeEnd'] = '12/31'
dfground['in_AllocationTimeframeStart'] = '01/01'
dfground['in_AllocationTypeCV'] = df_WR_GWSI['WELL_TYPE_']
dfground['in_BeneficialUseCategory'] = df_WR_GWSI['WATER_USE']
dfground['in_ExemptOfVolumeFlowPriority'] = 1

dfground

,in_ApplicableResourceTypeCV,in_WaterSourceUUID,in_WSGeometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_SiteUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_Geometry,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_MethodUUID,in_OrganizationUUID,in_PopulationServed,in_PowerType,in_PrimaryUseCategory,in_SiteUUID,in_VariableSpecificUUID,in_WaterAllocationNativeURL,in_WaterSourceUUID
0,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,PINAL,NaN,NaN,NaN,NaN,NaN,32.951858,-111.814054,NaN,NaN,POD,Unspecified,60000,NaN,NON-EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,60000,AR SACATON LLC,,NaN,12/31,01/01,NON-EXEMPT,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,MARICOPA,NaN,NaN,NaN,NaN,NaN,33.784972,-112.498131,NaN,NaN,POD,Unspecified,60001,NaN,NON-EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,NaN,60001,"SFI GRAND VISTA, LLC",,NaN,12/31,01/01,NON-EXEMPT,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,MARICOPA,NaN,NaN,NaN,NaN,NaN,33.500894,-113.170597,NaN,NaN,POD,Unspecified,82721,NaN,NON-EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,82721,"KEYES,G",,NaN,12/31,01/01,NON-EXEMPT,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,COCHISE,NaN,NaN,NaN,NaN,NaN,32.295123,-109.805511,NaN,NaN,POD,Unspecified,83226,NaN,EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,NaN,83226,TOLBERT COLEMAN,,NaN,12/31,01/01,EXEMPT,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,SANTA CRUZ,NaN,NaN,NaN,NaN,NaN,31.595636,-111.036613,NaN,NaN,POD,Unspecified,83578,NaN,NON-EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,83578,MYRON D CLARK,,NaN,12/31,01/01,NON-EXEMPT,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218117,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,PIMA,NaN,NaN,NaN,NaN,NaN,32.419050,-111.548384,NaN,NaN,POD,Unspecified,916136,NaN,MONITOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,916136,SILVER BELL MINING,,NaN,12/31,01/01,MONITOR,NaN,,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218118,Groundwater,NaN,NaN,NaN,NaN,Unspecified,NaN,groundwater/well,NaN,NaN,NaN,YAVAPAI,NaN,NaN,NaN,NaN,NaN,34.484237,-112.572941,NaN,NaN,POD,Unspecified,916137,NaN,NON-EXEMPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,916137,"DESIERTO FINO FARMS, INC.",,NaN,12/3

In [7]:
# Drop Duplicates
dfground = dfground.drop_duplicates()
dfground = dfground.replace(np.nan, '')  # Replaces NaN values with blank.

## Surface Water

In [8]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData/Surface_Water"
os.chdir(workingDir)

#Surfacewater Dataframes creation
SWR_fillings_input = "SWR_fillings_POD_input.csv"
df_SWRfill = pd.read_csv(SWR_fillings_input, encoding = "ISO-8859-1")

SWshed_input = "ADWR_Surface_Watersheds_POD_input.csv"
df_SWshed = pd.read_csv(SWshed_input, encoding = "ISO-8859-1")

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
# Need to restructure df_SWshed
# Single Flow_CFS as ANNUAL USE value.
# Seperate, then combine via comma Water Use.
# Retreive BenUse to single flow df.

df_SWshed_AU = df_SWshed.loc[df_SWshed["WATER USE"] == "ANNUAL USE"].reset_index()
df_SWshed_Ben = df_SWshed.loc[df_SWshed["WATER USE"] != "ANNUAL USE"].reset_index()

df_SWshed_Ben = df_SWshed_Ben.groupby('REG. NO').agg(lambda x: ','.join([str(elem) for elem in (list(set(x)))])).replace(np.nan, '').reset_index()
df_SWshed_Ben['BenUse'] = df_SWshed_Ben['WATER USE']
df_SWshed_Ben['REGNO'] = df_SWshed_Ben['REG. NO']

BenUseDict = pd.Series(df_SWshed_Ben.BenUse.values, index = df_SWshed_Ben.REGNO).to_dict()
def retrieveBenUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue
        try:
            outList = BenUseDict[String1]
        except:
            outList = ''
    return outList
df_SWshed_AU['BenUse'] = df_SWshed_AU.apply(lambda row: retrieveBenUse(row['REG. NO']), axis=1)

In [10]:
#Merge together
df_SWFill_SWshed = pd.merge(df_SWRfill, df_SWshed_AU, left_on='FILE_NO', right_on='REG. NO', how='left')

In [11]:
# Creating long and lat values from data.  
# Need to convert from UTM 12N to WGS 84.
# I believe AZ is consiered WGS 84 / UTM zone 12N - EPSG:32612.

from pyproj import Proj
myProj = Proj(proj='utm',zone=12, ellps='WGS84', preserve_units=False)
long, lat = myProj(df_SWFill_SWshed['X_UTMNAD83'].values, df_SWFill_SWshed['Y_UTMNAD83'].values, inverse=True)
df_SWFill_SWshed['in_Latitude'] = lat
df_SWFill_SWshed['in_Longitude'] = long
df_SWFill_SWshed = df_SWFill_SWshed.replace(np.nan, '')  # Replaces NaN values with blank.
df_SWFill_SWshed.head(3)

,ï»¿OID_,PROGRAM,APPNO,CONVNO,CERTNO,CERTSUFX,PERMITNO,PERMITSUF,PARENTAPP,POU_POD,STATUS_x,WATERSOURC,FILE_DATE,PRIOR_DATE,CONST_DT,USESTAT,RSRVNAME,HLDRNAME,HLDRCONAME,HLDRADDR,HLDRADDR2,HLDRCITY,HLDRZIP,HLDRZIP2,USES,WS_DESCR,CADASTRAL,CADASTRAL_,X_UTMNAD83,Y_UTMNAD83,FILE_NO,index,NAME,ART_WS_IDNO_FKFLD,ADDRESS,REG. NO,STATUS_y,PERMIT NO,CERT. NO,FILE DATE,SOURCE,PRIOR DATE,COUNTY,OWNER TYPE,WATERSHED,LEGAL,POU/POD,WATER USE,QUANTITY,QUANTITY_Val,QUANTITY_Unit,BenUse,in_Latitude,in_Longitude
0,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459844.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362
1,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459847.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POU,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362
2,170705,36,5457,0,,,,,36-5457,POD,ACTIVE - ACTIVE,1.5,9/30/1988,12/31/1883,,,,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,COWS / HORSES for STOCK for WILDLIFE using .4...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,36-5457.0,256769.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",36-5457.0,ACTIVE - ACTIVE,,,32416.0,1.5 DRAW,-5843.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362


In [12]:
# Creating AllocationFlow_CFS.
# Need to split string to value and units
# Need to convert based on string

# "Cubic Feet Per Second",

def CreateFlow_CFS(val, unit):
    if val == '' or pd.isnull(val):
        outVal = ""
    else:
        try:
            if unit == "Cubic Feet Per Second":
                outVal = float(val)
            else:
                outVal = ""
        except:
            outVal = ""
    return outVal

df_SWFill_SWshed['in_AllocationFlow_CFS'] = df_SWFill_SWshed.apply(lambda row: CreateFlow_CFS(row['QUANTITY_Val'], row['QUANTITY_Unit']), axis=1)
df_SWFill_SWshed.head(3)

,ï»¿OID_,PROGRAM,APPNO,CONVNO,CERTNO,CERTSUFX,PERMITNO,PERMITSUF,PARENTAPP,POU_POD,STATUS_x,WATERSOURC,FILE_DATE,PRIOR_DATE,CONST_DT,USESTAT,RSRVNAME,HLDRNAME,HLDRCONAME,HLDRADDR,HLDRADDR2,HLDRCITY,HLDRZIP,HLDRZIP2,USES,WS_DESCR,CADASTRAL,CADASTRAL_,X_UTMNAD83,Y_UTMNAD83,FILE_NO,index,NAME,ART_WS_IDNO_FKFLD,ADDRESS,REG. NO,STATUS_y,PERMIT NO,CERT. NO,FILE DATE,SOURCE,PRIOR DATE,COUNTY,OWNER TYPE,WATERSHED,LEGAL,POU/POD,WATER USE,QUANTITY,QUANTITY_Val,QUANTITY_Unit,BenUse,in_Latitude,in_Longitude,in_AllocationFlow_CFS
0,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459844.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,
1,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459847.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POU,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,
2,170705,36,5457,0,,,,,36-5457,POD,ACTIVE - ACTIVE,1.5,9/30/1988,12/31/1883,,,,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,COWS / HORSES for STOCK for WILDLIFE using .4...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,36-5457.0,256769.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",36-5457.0,ACTIVE - ACTIVE,,,32416.0,1.5 DRAW,-5843.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,


In [13]:
# Creating AllocationVolume_AF.
# Need to split string to value and units
# Need to convert value based on unit,

"Acre-Feet Per Annum",
"Acre-Feet",
"Acre-Feet Total",
"ACRES",
"CFT - Cubic Feet Total",
"Feet",
"Gallons",
"Gallons Per Annum",
"Miners Inches Per Annum",
"MIT - Miners Inches Total",

def CreateVolume_AF(val, unit):
    if val == '' or pd.isnull(val):
        outVal = ""
    else:
        try:
            if unit == "Acre-Feet Per Annum":
                val = float(val)
                outVal = val
            elif unit == "Acre-Feet":
                outVal = float(val)
            elif unit == "Acre-Feet Total":
                outVal = float(val)
            elif unit == "ACRES":
                outVal = float(val)
            elif unit == "CFT - Cubic Feet Total":
                val = float(val)
                outVal = val / (43559.9)  # Cubic Feet to AF
            elif unit == "Feet":
                val = float(val)
                outVal = val / (43559.9)  # Cubic Feet to AF
            elif unit == "Gallons":
                val = float(val)
                outVal = val / (325851)  # Gallons to AF
            elif unit == "Gallons Per Annum":
                val = float(val)
                outVal = val / (325851)  # Gallons to AF
            elif unit == "Miners Inches Per Annum":
                val = float(val)
                outVal = val * (0.055214457974269576)  # Miners Inches to AF
            elif unit == "MIT - Miners Inches Total":
                val = float(val)
                outVal = val * (0.055214457974269576)  # Miners Inches to AF
            else:
                outVal = ""
        except:
            outVal = ""
    return outVal

df_SWFill_SWshed['in_AllocationVolume_AF'] = df_SWFill_SWshed.apply(lambda row: CreateVolume_AF(row['QUANTITY_Val'], row['QUANTITY_Unit']), axis=1)
df_SWFill_SWshed.head(3)

,ï»¿OID_,PROGRAM,APPNO,CONVNO,CERTNO,CERTSUFX,PERMITNO,PERMITSUF,PARENTAPP,POU_POD,STATUS_x,WATERSOURC,FILE_DATE,PRIOR_DATE,CONST_DT,USESTAT,RSRVNAME,HLDRNAME,HLDRCONAME,HLDRADDR,HLDRADDR2,HLDRCITY,HLDRZIP,HLDRZIP2,USES,WS_DESCR,CADASTRAL,CADASTRAL_,X_UTMNAD83,Y_UTMNAD83,FILE_NO,index,NAME,ART_WS_IDNO_FKFLD,ADDRESS,REG. NO,STATUS_y,PERMIT NO,CERT. NO,FILE DATE,SOURCE,PRIOR DATE,COUNTY,OWNER TYPE,WATERSHED,LEGAL,POU/POD,WATER USE,QUANTITY,QUANTITY_Val,QUANTITY_Unit,BenUse,in_Latitude,in_Longitude,in_AllocationFlow_CFS,in_AllocationVolume_AF
0,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459844.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,,0.49
1,61513,38,94570,0,,,,,,POD,ACTIVE - ACTIVE,1.5,6/27/1979,12/31/1968,12/31/1968 0:00,,#1 1/2 TANK,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,for WILDLIFE using .49 AFA for ANNUAL USE23 f...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,38-94570.0,459847.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",38-94570.0,ACTIVE - ACTIVE,,,29033.0,1.5 DRAW,25203.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POU,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,,0.49
2,170705,36,5457,0,,,,,36-5457,POD,ACTIVE - ACTIVE,1.5,9/30/1988,12/31/1883,,,,AZ STATE LAND DEPT,,1616 W ADAMS ST,,PHOENIX,85007,,COWS / HORSES for STOCK for WILDLIFE using .4...,SAN PEDRO RIVER,D09014011AD0,D09014011AD0,512718.1993,3614317.878,36-5457.0,256769.0,AZ STATE LAND DEPT,36.0,"HEIDE KOCSIS 1616 W ADAMS ST PHOENIX, AZ 85007",36-5457.0,ACTIVE - ACTIVE,,,32416.0,1.5 DRAW,-5843.0,PINAL,STATE,SAN PEDRO RIVER,SE NE 11 9S 14E,POD,ANNUAL USE,.49 Acre-Feet Per Annum,0.49,Acre-Feet Per Annum,"WILDLIFE,STOCK",32.666443,-110.864362,,0.49


In [14]:
# Create output dataframe for ground water
columnslist = [
    ### Method Info ###
    "in_ApplicableResourceTypeCV",
    
    ### Water Source Info ###
    "in_WaterSourceUUID",
    "in_WSGeometry",
    "in_GNISFeatureNameCV",
    "in_WaterQualityIndicatorCV",
    "in_WaterSourceName",
    "in_WaterSourceNativeID",
    "in_WaterSourceTypeCV",
    
    ### Site Info ###
    "in_SiteUUID",
    "in_CoordinateAccuracy",
    "in_CoordinateMethodCV",
    "in_County",
    "in_EPSGCodeCV",
    "in_Geometry",
    "in_GNISCodeCV",
    "in_HUC12",
    "in_HUC8",
    "in_Latitude",
    "in_Longitude",
    "in_NHDNetworkStatusCV",
    "in_NHDProductCV",
    "in_PODorPOUSite",
    "in_SiteName",
    "in_SiteNativeID",
    "in_SitePoint",
    "in_SiteTypeCV",
    "in_StateCV",
    "in_USGSSiteID",
    
    ### AllocationAmount_fact Info ###
    "in_AllocationApplicationDate",
    "in_AllocationAssociatedConsumptiveUseSiteIDs",
    "in_AllocationAssociatedWithdrawalSiteIDs",
    "in_AllocationBasisCV",
    "in_AllocationChangeApplicationIndicator",
    "in_AllocationCommunityWaterSupplySystem",
    "in_AllocationCropDutyAmount",
    "in_AllocationExpirationDate",
    "in_AllocationFlow_CFS",
    "in_AllocationLegalStatusCV",
    "in_AllocationNativeID",
    "in_AllocationOwner",
    "in_AllocationPriorityDate",
    "in_AllocationSDWISIdentifierCV",
    "in_AllocationTimeframeEnd",
    "in_AllocationTimeframeStart",
    "in_AllocationTypeCV",
    "in_AllocationVolume_AF",
    "in_BeneficialUseCategory",
    "in_CommunityWaterSupplySystem",
    "in_CropTypeCV",
    "in_CustomerTypeCV",
    "in_DataPublicationDate",
    "in_DataPublicationDOI",
    "in_ExemptOfVolumeFlowPriority",
    "in_GeneratedPowerCapacityMW",
    "in_IrrigatedAcreage",
    "in_IrrigationMethodCV",
    "in_LegacyAllocationIDs",
    "in_MethodUUID",
    "in_OrganizationUUID",
    "in_PopulationServed",
    "in_PowerType",
    "in_PrimaryUseCategory",
    "in_SiteUUID",
    "in_VariableSpecificUUID",
    "in_WaterAllocationNativeURL",
    "in_WaterSourceUUID"
]

df_Surface = pd.DataFrame(columns=columnslist, index=df_SWFill_SWshed.index)

In [15]:
# For creating WaterSourceName
def assignWaterSourceName(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        strVal = str(colrowValue)
        outList = strVal.strip()
    return outList

# For creating SiteNativeID
def assignSiteNativeID(colrowValue):
    strVal = str(colrowValue)
    strVal = strVal.strip()
    if strVal == '' or pd.isnull(strVal):
        outList = "Unspecified"
    else:
        outList = strVal
    return outList

#############################################################################################
#Method
df_Surface['in_ApplicableResourceTypeCV'] = 'Surface Water'

#WaterSource
df_Surface['in_WaterSourceTypeCV'] = 'Surface Water'
df_Surface['in_WaterSourceName'] = df_SWFill_SWshed.apply(lambda row: assignWaterSourceName(row['WATERSOURC']), axis=1)
                                    
#Site
df_Surface['in_County'] = df_SWFill_SWshed['COUNTY']
df_Surface['in_Latitude'] = df_SWFill_SWshed['in_Latitude']
df_Surface['in_Longitude'] = df_SWFill_SWshed['in_Longitude']
df_Surface['in_PODorPOUSite'] = "POD"
df_Surface['in_SiteName'] = "Unspecified"
df_Surface['in_SiteNativeID'] = df_SWFill_SWshed.apply(lambda row: assignSiteNativeID(row['CADASTRAL']), axis=1)
df_Surface['in_SiteTypeCV'] = "Unspecified"

#AllocationAmount_fact
df_Surface['in_AllocationFlow_CFS'] = df_SWFill_SWshed['in_AllocationFlow_CFS']
df_Surface['in_AllocationVolume_AF'] = df_SWFill_SWshed['in_AllocationVolume_AF']
df_Surface['in_AllocationLegalStatusCV'] = df_SWFill_SWshed['STATUS_x']
df_Surface['in_AllocationNativeID'] = df_SWFill_SWshed['FILE_NO']
df_Surface['in_AllocationOwner'] = df_SWFill_SWshed['HLDRNAME']
df_Surface['in_AllocationTimeframeEnd'] = '12/31'
df_Surface['in_AllocationTimeframeStart'] = '01/01'
df_Surface['in_AllocationPriorityDate'] = df_SWFill_SWshed['PRIOR_DATE']
df_Surface['in_BeneficialUseCategory'] = df_SWFill_SWshed['BenUse']
df_Surface['in_ExemptOfVolumeFlowPriority'] = 0

In [16]:
# Drop Duplicates
df_Surface = df_Surface.drop_duplicates()

## Concatenate and Export Outputs

In [17]:
# Concatenate
frames = [dfground, df_Surface]
dfout = pd.concat(frames)

In [18]:
# OwnerClassification

OCDict = {
"US AIR FORCE" : "Army (USA)",
"US AIR FORCE - AFCEC/CZOM" : "Army (USA)",
"US AIR FORCE (ASC)" : "Army (USA)",
"US AIR FORCE LUKE AFB 56 RMO" : "Army (USA)",
"US AIR FORCE PLANT #44" : "Army (USA)",
"US AIR FORCE PLANT NO. 44" : "Army (USA)",
"US AIR FORCE(ASC/EMP)" : "Army (USA)",
"US AIR FORCE(ASC/EMR)" : "Army (USA)",
"US AIR FORCE," : "Army (USA)",
"US AIR FORCE, 56FW/RMO" : "Army (USA)",
"US AIR FORCE/ RAYTHEON" : "Army (USA)",
"US AIR FORCE/ASC/EMR," : "Army (USA)",
"US AIR FOREC/ASC/EMR," : "Army (USA)",
"US AIRFORCE" : "Army (USA)",
"US AIRFORCE DOD" : "Army (USA)",
"US ARMY COE," : "Army (USA)",
"US ARMY CORP ENGRS," : "Army (USA)",
"US ARMY CORP OF ENGINEER, LA DIST" : "Army (USA)",
"US ARMY CORP OF ENGINEERS, LA DIST" : "Army (USA)",
"US ARMY CORP OF," : "Army (USA)",
"US ARMY CORPS OF ENGINEERS" : "Army (USA)",
"US ARMY ENGINEER DIS," : "Army (USA)",
"US ARMY FT HUACHUCA," : "Army (USA)",
"US ARMY GARRISON YUMA PROVING GROUND" : "Army (USA)",
"US ARMY GARRISON YUMA PROVING GROUNDS" : "Army (USA)",
"US ARMY GARRISON, YUMA PROVING GROUND" : "Army (USA)",
"US ARMY PROVING GRND," : "Army (USA)",
"US ARMY YOUM PROVIDE," : "Army (USA)",
"US ARMY YUMA PROVIDE," : "Army (USA)",
"US ARMY YUMA PROVING GROUND" : "Army (USA)",
"US ARMY YUMA PROVING," : "Army (USA)",
"US ARMY," : "Army (USA)",
"US ARMY-YUMA PROVING," : "Army (USA)",
"US DEPT OF ARMY," : "Army (USA)",
"US MARINE CORP STAION" : "Army (USA)",
"US MARINE CORP STATION" : "Army (USA)",
"US MARINE CORP," : "Army (USA)",
"US MARINE CORPS" : "Army (USA)",
"US MARINE CORPS AIR STATION" : "Army (USA)",
"US MARINE CORPS AIR STATION YUMA" : "Army (USA)",
"US MARINE CORPS," : "Army (USA)",
"US MARINE CORPS, US MARINE CORPS" : "Army (USA)",
"US MARINE CORPS/MARINE CORPS AIR STATION" : "Army (USA)",
"US MARSHALLS OFFICE," : "Army (USA)",
"US NAVY/MCAS YUMA," : "Army (USA)",
"USACE" : "Army (USA)",
"USACE, LA DISTRICT (ATTN: DOUG DAHNCKE)" : "Army (USA)",
"USACE-CESPI-ED-GG" : "Army (USA)",
"USACE-CESPL-ED-GG" : "Army (USA)",
"USAF AERONAUTICAL SYSTEMS CENTER" : "Army (USA)",
"USAF PLANT 44" : "Army (USA)",
"USAF REAL PROPERTY AGENCY" : "Army (USA)",
"USAG FORT HUACHUCA" : "Army (USA)",
"USAG FT HUACHUCA" : "Army (USA)",
"USAG, FORT HUACHUCA" : "Army (USA)",
"US BUREAU OF LAND MANAGEMENT" : "Bureau of Land Management (USBLM)",
"US BUREAU OF LAND MANAGMENT" : "Bureau of Land Management (USBLM)",
"US BUREAU OF RECLAMATION" : "Bureau of Land Management (USBLM)",
"US BUREAU OF RECLAMATION-YUMA" : "Bureau of Land Management (USBLM)",
"USBOR" : "Bureau Reclamation (USBR)",
"US DEPARTMENT OF AGRICULTURE" : "Department of Agriculture (USDA)",
"US DEPT OF AGRICULTURE SIERRA VISTA RANGER STATION" : "Department of Agriculture (USDA)",
"USDA - APACHE-SITGREAVES NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA - FOREST SERVICE" : "Department of Agriculture (USDA)",
"USDA APACHE SITGRAVES NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA APACHE SITGREAVES NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE" : "Department of Agriculture (USDA)",
"USDA- FOREST SERVICE" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE - COCONINO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE - CORONADO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE CORONADO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE CORONADO NF" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE PNF CV RANGER DISTRICT" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE SW REGION R3" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE TONTO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE, APACHE-SITGREAVES NF" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE, CORONADO NAT'L FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE, PRESCOTT NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SERVICE/COCONINO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SVC," : "Department of Agriculture (USDA)",
"USDA FOREST SVC/CORONADO NATL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST SVCS CORONADO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA FOREST," : "Department of Agriculture (USDA)",
"USDA NATIONAL FOREST SERVICE" : "Department of Agriculture (USDA)",
"USDA PRESCOTT NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA TONTO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA- US FOREST SERVICE" : "Department of Agriculture (USDA)",
"USDA, FOREST SERVICE, KAIBAB NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA, TONTO NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA-ARS" : "Department of Agriculture (USDA)",
"USDAG" : "Department of Agriculture (USDA)",
"USDA-PRESCOTT NATIONAL FOREST" : "Department of Agriculture (USDA)",
"USDA-TONTO NATIONAL FOREST SERVICE" : "Department of Agriculture (USDA)",
"US DEPARTMENT OF THE INTERIOR" : "Department of the Interior (USDI)",
"US DEPT OF INTERIOR" : "Department of the Interior (USDI)",
"US DEPT OF INTERIOR," : "Department of the Interior (USDI)",
"US DEPT OF INTERIOR, NAT PARK SERVICE" : "Department of the Interior (USDI)",
"US DEPT OF THE INTERIOR FISH AND WILDLIFE SERVICE" : "Department of the Interior (USDI)",
"USDI NATL PARK SERV," : "Department of the Interior (USDI)",
"USDI NATL PARK SERV.," : "Department of the Interior (USDI)",
"USDI, BLM-LOWER SONORAN FIELD OFFICE" : "Department of the Interior (USDI)",
"USDI, BUREAU OF LAND MGT. KINGMAN FIELD OFFICE" : "Department of the Interior (USDI)",
"US E.P.A. REGION 9," : "Environmental Protection Agency (USEPA)",
"US ENVIRONMENTAL PROTECTION AGENCY" : "Environmental Protection Agency (USEPA)",
"US ENVIRONMENTAL PROTECTION AGENCY REG 9" : "Environmental Protection Agency (USEPA)",
"US ENVIRONMENTAL PROTECTION AGENCY, REGION 9" : "Environmental Protection Agency (USEPA)",
"US ENVIRONMENTAL PROTECTION AGENCY/REGION 9" : "Environmental Protection Agency (USEPA)",
"US EPA" : "Environmental Protection Agency (USEPA)",
"US EPA REGIOB 1X (SFD6-2)" : "Environmental Protection Agency (USEPA)",
"US EPA REGION 1X (SFD 6-2)" : "Environmental Protection Agency (USEPA)",
"US EPA REGION 1X (SFD6-2)" : "Environmental Protection Agency (USEPA)",
"US EPA REGION 1X," : "Environmental Protection Agency (USEPA)",
"US EPA REGION 9" : "Environmental Protection Agency (USEPA)",
"US EPA REGION 9," : "Environmental Protection Agency (USEPA)",
"US EPA REGION IX" : "Environmental Protection Agency (USEPA)",
"US EPA," : "Environmental Protection Agency (USEPA)",
"US EPA, REGION 9" : "Environmental Protection Agency (USEPA)",
"US EPA-REGION IX," : "Environmental Protection Agency (USEPA)",
"US DEPARTMENT OF FISH & WILDLIFE" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILD LIFE SERVICES" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE SAN BERNARDINO NWR" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE SERVICE" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE SERVICE - HAVASU" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE SERVICE, DIV. OF ENG., ATTN: CHARLES CALD" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE SERVICES" : "Fish and Wildlife Service (USFWS)",
"US FISH & WILDLIFE," : "Fish and Wildlife Service (USFWS)",
"US FISH AND WILDLIFE SERVICE" : "Fish and Wildlife Service (USFWS)",
"US FISH AND WILDLIFE SERVICE - SAN BERNARDINO NWR" : "Fish and Wildlife Service (USFWS)",
"US FISH AND WILDLIFE SERVICES" : "Fish and Wildlife Service (USFWS)",
"US FISH AND WILDLIFE," : "Fish and Wildlife Service (USFWS)",
"US FOREST SERVICE" : "Forest Service (USFS)",
"US FOREST SERVICE - - KAIBAB NATIONAL FOREST" : "Forest Service (USFS)",
"US FOREST SERVICE - TONTO NATIONAL FOREST" : "Forest Service (USFS)",
"US FOREST SERVICE RED ROCK DIST" : "Forest Service (USFS)",
"US FOREST SERVICE," : "Forest Service (USFS)",
"US FOREST SERVICE, APACHE-SITGREAVES NFS" : "Forest Service (USFS)",
"US FOREST SERVICE, ATTN: SHARRON HANSEN" : "Forest Service (USFS)",
"US FWS, IMPERIAL NWR," : "Forest Service (USFS)",
"US\ co APACHE-SITGRAVES NATIONAL FORESTS" : "Forest Service (USFS)",
"US GENERAL SERVICES" : "Geological Survey (USGS)",
"US GENERAL SERVICES," : "Geological Survey (USGS)",
"US GEOLOGICAL SURVEY" : "Geological Survey (USGS)",
"US GEOLOGICAL SURVEY," : "Geological Survey (USGS)",
"US ASSET MANAGEMENT, LLC" : "United States of America",
"US BANKCORP" : "United States of America",
"US BORAX & CHEMICAL," : "United States of America",
"US BORAX INC" : "United States of America",
"US BORAX INC," : "United States of America",
"US BORAX," : "United States of America",
"US BORDER PATROL" : "United States of America",
"US BORDER PATROL - NOGALES DIVISION" : "United States of America",
"US BORDER PATROL," : "United States of America",
"US CUSTOMS & BORDER PROTECTION" : "United States of America",
"US CUSTOMS AND BORDER PATROL" : "United States of America",
"US CUSTOMS SERVICE," : "United States of America",
"US DEPARTMENT OF HOMELAND SECURITY" : "United States of America",
"US DEVELOPMENT LAND LLC" : "United States of America",
"US DOE - WESTERN AREA POWER ADMINISTRATION" : "United States of America",
"US DOE WESTERN AREA POWER ADMIN" : "United States of America",
"US GENERAL SERVICE ADMINISTRATION" : "United States of America",
"US GENERAL SERVICES ADMINISTRATION, ATTN: SHEILA WILLIAMS" : "United States of America",
"US GOVT AFBCA/OC-S" : "United States of America",
"US GOVT AFBCA/OL-S," : "United States of America",
"US GOVT," : "United States of America",
"US HOME" : "United States of America",
"US HOME CORP" : "United States of America",
"US HOMES CORPORATION" : "United States of America",
"US INTERNATIONAL BOUNDARY AND WATER COMMISSION" : "United States of America",
"US LIFE TITLE CO," : "United States of America",
"US POST OFFICE," : "United States of America",
"US POSTAL SERVICE" : "United States of America",
"US POSTAL SERVICE," : "United States of America",
"US POWER SYSTEMS," : "United States of America",
"US RECYCLING," : "United States of America",
"US RICH HILL MINERAL," : "United States of America",
"US SMALL BUS ADMIN," : "United States of America",
"US TANK MGMT.," : "United States of America",
"US TONTO NATIONAL," : "United States of America",
"US WEST COMMUNICTN," : "United States of America",
"US WEST MT STATES," : "United States of America",
"US/USDAFS, CORONADO," : "United States of America",
"USA" : "United States of America",
"USA C/O DEPARTMENT OF HOMELAND SECURITY - CUSTOMS AND BORDER" : "United States of America",
"USA NATIONAL PARK SERVICE" : "United States of America",
"USA NATL PARK SERV," : "United States of America",
"USA WASTE OF ARIZONA," : "United States of America",
"USA, NATIONAL PARK SERVICE WATER RIGHTS BRANCH" : "United States of America",
"USA/ TONTO NATL FRST," : "United States of America",
"USA-BOR" : "United States of America"}

CommList = [
"CO",
"CORP",
"ENTRERPISE",
"INC",
"L.L.C.",
"LLC",
"PROPERTIES",
"RANCH"]

def CreateOwnerClassification(val):
    if val == '' or pd.isnull(val):
        outString = "Unspecified"
    else:
        String1 = val.strip()
        if String1 in OCDict:
            outString = OCDict[String1]
        elif any(word in String1 for word in CommList):
            outString = "Company"
        else:
            outString = "Privately Owned"
    return outString

dfout['OwnerClassification'] = dfout.apply(lambda row: CreateOwnerClassification(row['in_AllocationOwner']), axis=1)
print(len(dfout))
dfout.head(3)

312412


,in_ApplicableResourceTypeCV,in_WaterSourceUUID,in_WSGeometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_SiteUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_Geometry,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_MethodUUID,in_OrganizationUUID,in_PopulationServed,in_PowerType,in_PrimaryUseCategory,in_SiteUUID,in_VariableSpecificUUID,in_WaterAllocationNativeURL,in_WaterSourceUUID,OwnerClassification
0,Groundwater,,,,,Unspecified,,groundwater/well,,,,PINAL,,,,,,32.951858,-111.814054,,,POD,Unspecified,60000,,NON-EXEMPT,,,,,,,,,,,0,,60000,AR SACATON LLC,,,12/31,01/01,NON-EXEMPT,,,,,,,,1,,,,,,,,,,,,,,Company
1,Groundwater,,,,,Unspecified,,groundwater/well,,,,MARICOPA,,,,,,33.784972,-112.498131,,,POD,Unspecified,60001,,NON-EXEMPT,,,,,,,,,,,200,,60001,"SFI GRAND VISTA, LLC",,,12/31,01/01,NON-EXEMPT,,,,,,,,1,,,,,,,,,,,,,,Company
2,Groundwater,,,,,Unspecified,,groundwater/well,,,,MARICOPA,,,,,,33.500894,-113.170597,,,POD,Unspecified,82721,,NON-EXEMPT,,,,,,,,,,,0,,82721,"KEYES,G",,,12/31,01/01,NON-EXEMPT,,,,,,,,1,,,,,,,,,,,,,,Privately Owned


In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

in_ApplicableResourceTypeCV                      object
in_WaterSourceUUID                               object
in_WSGeometry                                    object
in_GNISFeatureNameCV                             object
in_WaterQualityIndicatorCV                       object
in_WaterSourceName                               object
in_WaterSourceNativeID                           object
in_WaterSourceTypeCV                             object
in_SiteUUID                                      object
in_CoordinateAccuracy                            object
in_CoordinateMethodCV                            object
in_County                                        object
in_EPSGCodeCV                                    object
in_Geometry                                      object
in_GNISCodeCV                                    object
in_HUC12                                         object
in_HUC8                                          object
in_Latitude                                     

In [20]:
#Exporting to Finished File
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Arizona/WaterAllocation/RawInputData"
os.chdir(workingDir)

dfout.to_csv('P_ArizonaMaster.csv', index=False)  # The output